In [7]:
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
import numpy as np

# Przygotowanie danych
def load_data(load_function):
    """
    X_train (numpy.ndarray) - zbiór danych treningowych
    X_test (numpy.ndarray) - zbiór danych testowych
    y_train (numpy.ndarray) - etykiety danych treningowych
    y_test (numpy.ndarray) - etykiety danych testowych
    class_num (int) - liczba klas
    """
    
    # Wczytanie danych
    data = load_function()
    X = data.data
    y = data.target
    
    # Przekształcenie etykiet na format one-hot ( 1 lub 0 )
    y = pd.Categorical(y)
    y = pd.get_dummies(y)
    class_num = y.shape[1]
    
    # Podział na zbiór treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Skalowanie danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test, class_num

# Funkcja do budowy modelu
def build_model(num_layers, neurons, activation, optimizer, learning_rate):
    model = Sequential()
    model.add(Input(shape=(X.shape[1],)))
    for _ in range(num_layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(class_num, activation="softmax"))
    
    # Wybór optymalizatora
    if optimizer == "adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == "rmsprop":
        opt = RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("Nieobsługiwany optymalizator")
    
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Parametry do testowania
param_grid = {
    "num_layers": [2, 3],  # Liczba warstw
    "neurons": [64, 128],  # Liczba neuronów w warstwie
    "activation": ["relu", "tanh"],  # Funkcja aktywacji
    "optimizer": ["adam", "sgd"],  # Optymalizator
    "learning_rate": [0.01, 0.001],  # Prędkość nauczania
    "batch_size": [32, 64],  # Wielkość batcha
    "epochs": [50, 100]  # Liczba epok
}

# Wyniki
results = []

# Przeszukiwanie siatkowe

X_train, X_test, y_train, y_test, class_num = load_data(load_digits)

for num_layers in param_grid["num_layers"]:
    for neurons in param_grid["neurons"]:
        for activation in param_grid["activation"]:
            for optimizer in param_grid["optimizer"]:
                for learning_rate in param_grid["learning_rate"]:
                    for batch_size in param_grid["batch_size"]:
                        for epochs in param_grid["epochs"]:
                            print(f"Testowanie: num_layers={num_layers}, neurons={neurons}, activation={activation}, "
                                  f"optimizer={optimizer}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
                            
                            # Budowanie modelu
                            model = build_model(num_layers, neurons, activation, optimizer, learning_rate)
                            
                            # Trenowanie modelu
                            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_split=0.2)
                            
                            # Ewaluacja
                            y_pred = np.argmax(model.predict(X_test), axis=1)
                            y_true = np.argmax(y_test.values, axis=1)
                            accuracy = accuracy_score(y_true, y_pred)
                            
                            # Zapisanie wyników
                            results.append({
                                "num_layers": num_layers,
                                "neurons": neurons,
                                "activation": activation,
                                "optimizer": optimizer,
                                "learning_rate": learning_rate,
                                "batch_size": batch_size,
                                "epochs": epochs,
                                "accuracy": accuracy
                            })

# Konwersja wyników na DataFrame
import pandas as pd
results_df = pd.DataFrame(results)

# Wyświetlenie najlepszej konfiguracji
best_result = results_df.loc[results_df["accuracy"].idxmax()]
print("Najlepszy wynik:")
print(best_result)

# Wyświetlenie wyników w kolejności dokładności
print(results_df.sort_values(by="accuracy", ascending=False))


Testowanie: num_layers=2, neurons=64, activation=relu, optimizer=adam, learning_rate=0.01, batch_size=32, epochs=50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Testowanie: num_layers=2, neurons=64, activation=relu, optimizer=adam, learning_rate=0.01, batch_size=32, epochs=100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Testowanie: num_layers=2, neurons=64, activation=relu, optimizer=adam, learning_rate=0.01, batch_size=64, epochs=50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Testowanie: num_layers=2, neurons=64, activation=relu, optimizer=adam, learning_rate=0.01, batch_size=64, epochs=100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Testowanie: num_layers=2, neurons=64, activation=relu, optimizer=adam, learning_rate=0.001, batch_size=32, epochs=50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Testowanie: num_layers=2, neurons=64, activation=relu, optimizer=adam, learning_rate=0.001, batch_size=32, epochs=100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Testowanie: num_layers=2, neurons=64, activation=relu, optim

Najlepszy wynik:
num_layers              3
neurons               128
activation           relu
optimizer            adam
learning_rate        0.01
batch_size             64
epochs                 50
accuracy         0.983333
Name: 98, dtype: object
     num_layers  neurons activation optimizer  learning_rate  batch_size  \
98            3      128       relu      adam          0.010          64   
100           3      128       relu      adam          0.001          32   
3             2       64       relu      adam          0.010          64   
115           3      128       tanh      adam          0.010          64   
101           3      128       relu      adam          0.001          32   
..          ...      ...        ...       ...            ...         ...   
79            3       64       relu       sgd          0.001          64   
110           3      128       relu       sgd          0.001          64   
30            2       64       tanh       sgd          0.001          64   
14            2       64       relu       sgd          0.001          64   
78            3       64       relu       sgd          0.001          64   

     epochs  accuracy  
98       50  0.983333  
100      50  0.980556  
3       100  0.980556  
115     100  0.977778  
101     100  0.977778  
..      ...       ...  
79      100  0.700000  
110      50  0.675000  
30       50  0.650000  
14       50  0.486111  
78       50  0.422222  

[128 rows x 8 columns]